In [ ]:
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import pandas as pd
# --- 准备工作 ---

# 1. 定义一个计算两个经纬度点之间距离的函数 (Haversine a公式)
#    这是比默认欧式距离更适合经纬度的度量方式
def haversine_distance(p1, p2):
    """
    计算两个经纬度点之间的球面距离（单位：米）
    p1, p2: 两个点的坐标，格式为 [经度, 纬度]
    """
    R = 6371000  # 地球平均半径，单位为米
    
    lon1, lat1 = p1
    lon2, lat2 = p2
    
    # 将经纬度从度数转换为弧度
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)
    
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    distance = R * c
    return distance


def string_to_trajectory(input_string: str) -> np.ndarray:
    # 检查输入是否为空或仅包含空白字符
    if not input_string or not input_string.strip():
        # 返回一个空的 (0, 2) 形状的数组，表示空轨迹
        return np.empty((0, 2))

    # 存储处理后的坐标点
    points = []
    
    # 1. 使用 '#' 分割字符串，得到每个点的字符串列表
    point_strings = input_string.strip().split('#')
    
    for i, p_str in enumerate(point_strings):
        # 2. 对每个点的字符串，使用 ',' 分割经纬度
        coords = p_str.split(',')
        
        # 3. 检查分割后是否恰好是两个值
        if len(coords) != 2:
            raise ValueError(f"格式错误：在第 {i+1} 个点 '{p_str}' 中，坐标数量不为2。")
            
        try:
            # 4. 将字符串转换为浮点数
            lon = float(coords[0])
            lat = float(coords[1])
            points.append([lon, lat])
        except ValueError:
            # 如果转换失败，说明包含非数字字符
            raise ValueError(f"格式错误：在第 {i+1} 个点 '{p_str}' 中，包含无法转换为数字的坐标。")
            
    # 5. 将Python列表转换为NumPy数组
    trajectory = np.array(points)
    
    return trajectory


# 创建一个函数，将Numpy数组转换为scikit-mobility所需的Trajectory对象
def create_skmob_trajectory(np_array, traj_id):
    # scikit-mobility需要DataFrame格式的数据
    # 列名必须是 'lng', 'lat', 'traj_id'
    df = pd.DataFrame(np_array, columns=['lng', 'lat'])
    df['traj_id'] = traj_id
    
    # 从DataFrame创建TrajectoryDataFrame
    # crs='epsg:4326' 告诉库这是标准的经纬度坐标
    tdf = skmob.TrajDataFrame(df, latitude='lat', longitude='lng', traj_id='traj_id', crs='epsg:4326')
    
    # TrajectoryDataFrame可能包含多个轨迹，我们只返回第一个（也是唯一一个）
    return tdf.loc[traj_id]


string_a = "121.347,31.392#121.348,31.389#121.349,31.390#121.350,31.390#121.351,31.390#121.351,31.391#121.352,31.391#121.353,31.391#121.353,31.394#121.354,31.391#121.355,31.391#121.356,31.389#121.356,31.390#121.356,31.391#121.357,31.388#121.357,31.389"
trajectory_a = string_to_trajectory(string_a)
string_b = "121.489,31.270#121.489,31.271#121.490,31.270#121.490,31.271#121.491,31.271#121.492,31.270#121.492,31.271#121.493,31.269#121.493,31.270#121.494,31.268#121.494,31.269#121.495,31.269#121.495,31.270#121.496,31.270#121.497,31.270#121.497,31.271#121.498,31.272#121.499,31.272#121.499,31.273#121.500,31.273#121.500,31.274#121.501,31.274#121.502,31.273#121.502,31.274#121.503,31.273#121.504,31.273#121.504,31.274#121.505,31.274#121.505,31.275#121.506,31.275#121.506,31.276#121.506,31.277#121.507,31.277#121.507,31.278#121.507,31.279#121.508,31.279"
trajectory_b = string_to_trajectory(string_b)

traj_a = create_skmob_trajectory(trajectory_a, 1)
traj_b = create_skmob_trajectory(trajectory_b, 2)
# --- 计算弗雷歇距离 ---

# 1. 使用默认的欧几里得距离计算
#    这适用于普通XY坐标系，对于经纬度来说精度不高，但可以快速比较
frechet_dist_ab = traj_a.distance(traj_b, metric='frechet')

print("使用 scikit-mobility 计算：")
print(f"轨迹A和B的弗雷歇距离: {frechet_dist_ab:.2f} 米")

NameError: name 'skmob' is not defined